# 오픈 데이터 추가
- https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=577 에서 데이터 로컬에 저장 후 실행

In [1]:
import json
import numpy as np
from tqdm import tqdm

In [2]:
def read_json(file_path: str):
    with open(file_path) as f:
        dic = json.load(f)
    return dic

In [26]:
# 추출형 데이터 불러오기 
train_extraction_extracion = read_json("/dev/shm/open_data/training/TL_span_extraction.json")
validation_extraction_extracion = read_json("/dev/shm/open_data/validation/VL_span_extraction.json")

In [27]:
train_extraction_extracion.keys()

dict_keys(['Dataset', 'data'])

In [28]:
len(train_extraction_extracion['data']), len(validation_extraction_extracion['data'])

(111967, 13997)

In [3]:
# 추론형 데이터 불러오기
train_inference_extracion = read_json("/dev/shm/open_data/training/TL_span_inference.json")
validation_inference_extracion = read_json("/dev/shm/open_data/validation/VL_span_inference.json")

In [4]:
len(train_inference_extracion['data']), len(validation_inference_extracion['data'])

(15992, 1999)

# Dataset Filtering

In [5]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

/data/ephemeral/home/sh/level2-mrc-nlp-08/.venv_sh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_filter_lst = []
ids = 0
for row in tqdm(train_inference_extracion['data'], total=len(train_inference_extracion['data'])):
    # print(row)
    doc_id = row['doc_id']
    doc_title = row['doc_title']
    doc_class = row['doc_class']['code']
    context = row['paragraphs'][0]['context']
    # context 토큰 1500 초과 제거
    if len(tokenizer.encode(context)) > 1500:
        continue

    question_answer_lst = row['paragraphs'][0]['qas']
    # question 토큰 35 초과 제거 / context 토큰 1500 초과 제거 / answer 토큰 20 초과 제거
    for qas in question_answer_lst:
        question = qas['question']
        # question 토큰 35 초과 제거
        if len(tokenizer.encode(question)) > 35:
            continue
        
        text = qas['answers']['text']
        # answer 토큰 20 초과 제거
        if len(tokenizer.encode(text)) > 20:
            continue
        
        answer_start = qas['answers']['answer_start']
        ids+=1
        train_filter_lst.append([ids, doc_id, doc_title, question, context, {'answer_start':[answer_start], 'text':[text]}])


  0%|          | 0/15992 [00:00<?, ?it/s]

100%|██████████| 15992/15992 [00:46<00:00, 342.73it/s]


In [8]:
import pandas as pd

In [9]:
'''
title	context	question	id	answers	document_id	__index_level_0__
0	미국 상원	미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...	대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?	mrc-1-000067	{'answer_start': [235], 'text': ['하원']}	18293.0	42.0
1	인사조직관리	'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...	현대적 인사조직관리의 시발점이 된 책은?	mrc-0-004397	{'answer_start': [212], 'text': ['《경영의 실제》']}	51638.0	2873.0
'''

columns = ['id', 'document_id', 'title', 'question', 'context', 'answers']
df_train = pd.DataFrame(train_filter_lst, columns=columns)

In [10]:
df_train

,id,document_id,title,question,context,answers
0,1,01400601.20210325223148001,"충북도 5843억 투입, 일자리 10만8800개 창출",창업 아이템 경진대회에 참가할 수 있으며 청춘잡담도 이용할 수 있는 대상이 누구야,충북도가 5843억원을 투입해 지역일자리 10만8000개 창출과 고용률 72% 달성...,"{'answer_start': [700], 'text': ['청년여성']}"
1,2,01400601.20210325223148001,"충북도 5843억 투입, 일자리 10만8800개 창출",청춘잡담을 이용할 수도 있는 창업 아이템 경진대회 참가 대상자는 어떤 사람들이야,충북도가 5843억원을 투입해 지역일자리 10만8000개 창출과 고용률 72% 달성...,"{'answer_start': [700], 'text': ['청년여성']}"
2,3,01400601.20210325175451001,공직유관단체장·기초의원 175명 재산 공개,도 공직자윤리위원회의 조사에 따르면 충남의 현직 지방 의원 중 자산이 가장 많은 사...,충남도 공직자윤리위원회는 도내 각 시ㆍ군의회 의원과 서산의료원장 등 도 관할 대상자...,"{'answer_start': [817], 'text': ['맹의석']}"
3,4,01400601.20210325175451001,공직유관단체장·기초의원 175명 재산 공개,충청남도 지방의회 의원 중에서 누가 재산이 가장 많은 걸로 도 공직자윤리위원회 조사...,충남도 공직자윤리위원회는 도내 각 시ㆍ군의회 의원과 서산의료원장 등 도 관할 대상자...,"{'answer_start': [817], 'text': ['맹의석']}"
4,5,01400601.20210325223159001,"솔밭중, 온라인 학교설명회",솔밭중이 한 해 동안 학교를 어떻게 이끌어갈 것이지 설명한 회의는 어느 플랫폼에서 열렸어,충북 청주 솔밭중은 25일 온라인 학교설명회와 학부모 총회를 열었다. \n\n202...,"{'answer_start': [145], 'text': ['유튜브']}"
...,...,...,...,...,...,...
31632,31633,01101101.20210415174300001,"노인 변비, 소화 문제 아니라 '신체 노쇠' 신호탄?",변비와 신체 노쇠에 대한 예방법을 말한 건 누구야,고령인에게 생기는 변비는 단순히 소화 문제를 넘어 평소에는 알아차리지 못했던 ‘신체...,"{'answer_start': [98], 'text': ['정희원']}"
31633,31634,01101101.20210415123111001,"원희룡, 김종인과 선 긋기 ""윤석열 국민의힘 안 간다? 본인도 모를 것""",김종인의 말에 반박한 사람이 나온 라디오 프로그램의 이름은 뭐야,내년 대선 출마를 시사한 원희룡 제주지사가 최근 김종인 전 국민의힘 비상대책위원장의...,"{'answer_start': [197], 'text': ['최경영의 최강시사']}"
31634,31635,01101101.20210415123111001,"원희룡, 김종인과 선 긋기 ""윤석열 국민의힘 안 간다? 본인도 모를 것""",어떤 라디오 프로그램에 김종인의 말에 반박한 사람이 나온 거야,내년 대선 출마를 시사한 원희룡 제주지사가 최근 김종인 전 국민의힘 비상대책위원장의...,"{'answer_start': [197], 'text': ['최경영의 최강시사']}"
31635,31636,01101101.20210415082251001,블락비 데뷔 10주년...깜짝 생방송 출동,팬클럽 이름이 BBC인 그룹은 어디를 통해 팬들에게 10주년 소감을 전했어,그룹 블락비(Block B)가 데뷔 10주년을 맞았다.\n지난 2011년 4월 15...,"{'answer_start': [462], 'text': ['세븐시즌스']}"


In [11]:
from datasets import load_from_disk
datasets = load_from_disk("../resources/data/train_dataset") # 기존 train dataset

In [12]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train, preserve_index=False)
validation_dataset=datasets['validation']

In [13]:
from datasets import DatasetDict

In [14]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'title', 'question', 'context', 'answers'],
        num_rows: 31637
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


In [15]:
dataset_dict.save_to_disk('../resources/data_aihub_train_dataset')

Saving the dataset (0/1 shards):  32%|███▏      | 10000/31637 [00:00<00:00, 94223.53 examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 42549.37 examples/s]
